<a href="https://colab.research.google.com/github/anandsai410/Blockchain-technologies-Assignment-I/blob/main/ORT_Manual_Apache%20Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import datetime
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk import punkt
import openpyxl
import pandas as pd
import numpy as np
import re
from datetime import datetime, date, time
import time
from dateutil.parser import parser
from dateutil.relativedelta import relativedelta
parser=parser()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
!pip install spark-nlp
!pip install pyspark

import sparknlp
from sparknlp.annotator import DocumentAssembler, DateMatcher, MultiDateMatcher
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline

spark = sparknlp.start()
spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 8.2 MB/s eta 0:00:00


In [4]:
df = pd.read_excel("Test_cases_DEC23.xlsx")
df

,Case,External Info,ORT manual,Updated by,Customer contact,Company,Source channel Task,State,Priority,Assigned to,Updated,Tags,CP_ext_inf
0,API user restriction​CS20230006607972,2023-12-20 12:09:32 - Rohit Panwar (I322661) (...,2023-12-20 15:00:35,I322661,Anitha Ponna (S0025152086),NaN,Chat,Resolved,3 - Medium,Rohit Panwar (I322661),2023-12-20 12:09:32,$ORT_Manual_OK,NaN
1,Users not imported from SF into LMS​CS20230006...,2023-12-15 04:12:16 - System (External Info)\n...,2023-12-15 15:00:31,I322661,Dominic Ponnuswamy (S0024832552),NaN,Portal,Closed,2 - High,Rohit Panwar (I322661),2023-12-18 10:35:06,"$ORT_Manual_OK, @Not_Release, IRT_METBY_QG",NaN
2,WFA shows no pending reqs but there are pendin...,2023-12-27 05:18:08 - System (External Info)\n...,2023-12-06 20:41:39,I501666,Mandy Birk (S0016457108),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:01:32,"$ORT_Manual_NOK, IRT_METBY_POC",NaN
3,Can't activate Onboarding 2.0 in the Productio...,2023-12-01 08:26:46 - Subba Reddy Venkata Takk...,2023-12-13 02:35:21,I501666,Alfonso Adeva (S0024115440),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-05 11:58:27,$ORT_Manual_NOK,NaN
4,Not able to add input control for background e...,2024-01-01 07:04:12 - Subba Reddy Venkata Takk...,2023-12-31 18:06:09,I501666,Aji Hussain (S0021762292),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:04:12,$ORT_Manual_NOK,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Received an error when importing a story​CS202...,2023-12-18 10:31:14 - Subba Reddy Venkata Takk...,2023-12-21 20:30:00,system,Kumi Mikami (S0024714497),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-28 08:52:48,$ORT_Manual_NOK,NaN
113,"Everytime I try to run a story report, the err...",2024-01-09 05:51:25 - System (External Info)\n...,2023-12-19 20:58:38,system,Park Sujeong (S0024711903),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-09 05:51:25,$ORT_Manual_NOK,NaN
114,ORD Report Error​CS20230006745117,2023-12-21 20:47:58 - Subba Reddy Venkata Takk...,2023-12-22 13:33:58,system,Jianliang Yin (S0019691286),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-30 09:34:40,$ORT_Manual_NOK,NaN
115,An error occurred while importing a Story repo...,2023-12-28 06:03:25 - Subba Reddy Venkata Takk...,2023-12-31 12:54:39,system,Napapatch Rakwong (S0025555256),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-29 08:57:03,$ORT_Manual_NOK,NaN


In [5]:
ext_inf=df['External Info']
dte=df['ORT manual']
df['dates']=''

In [6]:
pattern_SE = r'\<p>(.*?)\</p>'
#pattern_OA = r'\<br/>(.*?)\<br/>'

for i in range(0,len(df)):
  xstr=""
  x = re.findall(pattern_SE, df.iloc[i,1])
  #y = re.findall(pattern_OA, df.iloc[i,1])
  for j in x:
     xstr=xstr+". "+str(j)
  #for j in y:
     #xstr = xstr+". "+str(j)
  df.iloc[i,12]=xstr

In [7]:
df

,Case,External Info,ORT manual,Updated by,Customer contact,Company,Source channel Task,State,Priority,Assigned to,Updated,Tags,CP_ext_inf,dates
0,API user restriction​CS20230006607972,2023-12-20 12:09:32 - Rohit Panwar (I322661) (...,2023-12-20 15:00:35,I322661,Anitha Ponna (S0025152086),NaN,Chat,Resolved,3 - Medium,Rohit Panwar (I322661),2023-12-20 12:09:32,$ORT_Manual_OK,". Dear Anitha,. Thank you for your response. T...",
1,Users not imported from SF into LMS​CS20230006...,2023-12-15 04:12:16 - System (External Info)\n...,2023-12-15 15:00:31,I322661,Dominic Ponnuswamy (S0024832552),NaN,Portal,Closed,2 - High,Rohit Panwar (I322661),2023-12-18 10:35:06,"$ORT_Manual_OK, @Not_Release, IRT_METBY_QG",". Dear Dominic,. I have looked into the integr...",
2,WFA shows no pending reqs but there are pendin...,2023-12-27 05:18:08 - System (External Info)\n...,2023-12-06 20:41:39,I501666,Mandy Birk (S0016457108),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:01:32,"$ORT_Manual_NOK, IRT_METBY_POC",". Hi Mandy,. As requested plesae provide the d...",
3,Can't activate Onboarding 2.0 in the Productio...,2023-12-01 08:26:46 - Subba Reddy Venkata Takk...,2023-12-13 02:35:21,I501666,Alfonso Adeva (S0024115440),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-05 11:58:27,$ORT_Manual_NOK,". Hi Customer,. Corrected config related to er...",
4,Not able to add input control for background e...,2024-01-01 07:04:12 - Subba Reddy Venkata Takk...,2023-12-31 18:06:09,I501666,Aji Hussain (S0021762292),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:04:12,$ORT_Manual_NOK,. Solution provided:[code]. As of now closing ...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Received an error when importing a story​CS202...,2023-12-18 10:31:14 - Subba Reddy Venkata Takk...,2023-12-21 20:30:00,system,Kumi Mikami (S0024714497),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-28 08:52:48,$ORT_Manual_NOK,". Thanks for the patience Kumi,. Engineering t...",
113,"Everytime I try to run a story report, the err...",2024-01-09 05:51:25 - System (External Info)\n...,2023-12-19 20:58:38,system,Park Sujeong (S0024711903),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-09 05:51:25,$ORT_Manual_NOK,". Hi,. I am available for call in any feasible...",
114,ORD Report Error​CS20230006745117,2023-12-21 20:47:58 - Subba Reddy Venkata Takk...,2023-12-22 13:33:58,system,Jianliang Yin (S0019691286),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-30 09:34:40,$ORT_Manual_NOK,". Thanks, as of now closing the case. Looks li...",
115,An error occurred while importing a Story repo...,2023-12-28 06:03:25 - Subba Reddy Venkata Takk...,2023-12-31 12:54:39,system,Napapatch Rakwong (S0025555256),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-29 08:57:03,$ORT_Manual_NOK,. Solution provided:[code]. &nbsp;. Issue is f...,


In [42]:
# Step 1: Transforms raw texts to `document` annotation
document_assembler = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)
# Step 2: Extracts one date information from text
date = (
    DateMatcher()
    .setInputCols("document") \
    .setOutputCol("date") \
    .setOutputFormat("yyyy/MM/dd")
)

pipl = Pipeline(stages=[document_assembler, date])

for i in range(0,len(df)-1):
  dtestr=""
  texts = sent_tokenize(df.iloc[i,12])
  spark_df = spark.createDataFrame(texts, StringType()).toDF("text")
  result = pipl.fit(spark_df).transform(spark_df)
  resultdf = result.toPandas()
  for j in resultdf['date']:
    if j != []:
      for k in j:
        dtestr=dtestr+str(k.result)+", "
  df.iloc[i,13]=dtestr

In [43]:
df

,Case,External Info,ORT manual,Updated by,Customer contact,Company,Source channel Task,State,Priority,Assigned to,Updated,Tags,CP_ext_inf,dates
0,API user restriction​CS20230006607972,2023-12-20 12:09:32 - Rohit Panwar (I322661) (...,2023-12-20 15:00:35,I322661,Anitha Ponna (S0025152086),NaN,Chat,Resolved,3 - Medium,Rohit Panwar (I322661),2023-12-20 12:09:32,$ORT_Manual_OK,". Dear Anitha,. Thank you for your response. T...","2311/01/09, 2023/12/08, 2023/12/08, 2024/03/24..."
1,Users not imported from SF into LMS​CS20230006...,2023-12-15 04:12:16 - System (External Info)\n...,2023-12-15 15:00:31,I322661,Dominic Ponnuswamy (S0024832552),NaN,Portal,Closed,2 - High,Rohit Panwar (I322661),2023-12-18 10:35:06,"$ORT_Manual_OK, @Not_Release, IRT_METBY_QG",". Dear Dominic,. I have looked into the integr...","2024/03/24, 2023/12/11,"
2,WFA shows no pending reqs but there are pendin...,2023-12-27 05:18:08 - System (External Info)\n...,2023-12-06 20:41:39,I501666,Mandy Birk (S0016457108),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:01:32,"$ORT_Manual_NOK, IRT_METBY_POC",". Hi Mandy,. As requested plesae provide the d...",
3,Can't activate Onboarding 2.0 in the Productio...,2023-12-01 08:26:46 - Subba Reddy Venkata Takk...,2023-12-13 02:35:21,I501666,Alfonso Adeva (S0024115440),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-05 11:58:27,$ORT_Manual_NOK,". Hi Customer,. Corrected config related to er...",
4,Not able to add input control for background e...,2024-01-01 07:04:12 - Subba Reddy Venkata Takk...,2023-12-31 18:06:09,I501666,Aji Hussain (S0021762292),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:04:12,$ORT_Manual_NOK,. Solution provided:[code]. As of now closing ...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Received an error when importing a story​CS202...,2023-12-18 10:31:14 - Subba Reddy Venkata Takk...,2023-12-21 20:30:00,system,Kumi Mikami (S0024714497),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-28 08:52:48,$ORT_Manual_NOK,". Thanks for the patience Kumi,. Engineering t...","2024/01/12, 2024/03/24, 2024/03/24, 2024/01/12..."
113,"Everytime I try to run a story report, the err...",2024-01-09 05:51:25 - System (External Info)\n...,2023-12-19 20:58:38,system,Park Sujeong (S0024711903),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-09 05:51:25,$ORT_Manual_NOK,". Hi,. I am available for call in any feasible...","2024/03/24,"
114,ORD Report Error​CS20230006745117,2023-12-21 20:47:58 - Subba Reddy Venkata Takk...,2023-12-22 13:33:58,system,Jianliang Yin (S0019691286),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-30 09:34:40,$ORT_Manual_NOK,". Thanks, as of now closing the case. Looks li...",
115,An error occurred while importing a Story repo...,2023-12-28 06:03:25 - Subba Reddy Venkata Takk...,2023-12-31 12:54:39,system,Napapatch Rakwong (S0025555256),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-29 08:57:03,$ORT_Manual_NOK,. Solution provided:[code]. &nbsp;. Issue is f...,


In [46]:
print(df.iloc[1,13])

2024/03/24, 2023/12/11, 


In [ ]:
resultdf=result.toPandas()
import openpyxl
#resultdf.to_excel('spark.xlsx', sheet_name='dates form spark')